In [ ]:
# Notebook parameters. Values here are for development only and
# will be overridden when running via snakemake and papermill.

# General setup parameters.
atlas_id = "agam"
analysis_version = "dev"
cohorts_analysis = "20240924"
contigs = ["3L"]
dask_scheduler = "single-threaded"

# Other parameters.
sample_sets = "AG1000G-BF-A"
sample_query = None
min_cohort_size = 20
max_cohort_size = 50
h12_calibration_contig = "3L"

contig = "3L"

In [ ]:
from bokeh.io import output_notebook
from selection_atlas.setup import AtlasSetup
from selection_atlas.page_utils import AtlasPageUtils

# Initialise the atlas setup.
setup = AtlasSetup(
    atlas_id=atlas_id,
    analysis_version=analysis_version,
    cohorts_analysis=cohorts_analysis,
    contigs=contigs,
    dask_scheduler=dask_scheduler,
)

# Initialise page utils.
page_utils = AtlasPageUtils(setup=setup)

# N.B., do not add the "remove-output" tag to this cell!!! If you do,
# the bokeh javascript libraries will not get loaded in the generated
# HTML page. The call to output_notebook() injects javascript in the
# cell output which triggers the bokeh javascript libraries to be loaded
# in the page.
output_notebook(hide_banner=True)

# Chromosome Foo

The plot below shows {term}`selection signal`s discovered in any of the mosquito {term}`cohort`s analysed. Hover over a signal for more information about the {term}`cohort` in which the signal was found, including the mosquito species, collection location and date range. Click on a signal to see the underlying selection scan data.

In [ ]:
df_signals = page_utils.load_signals(contig=contig)

page_utils.plot_signals(df=df_signals, contig=contig)

In [ ]:
def style_signals_table(
    df_signals,
):
    df_signals = df_signals[
        [
            "contig",
            "focus_pstart",
            "focus_pstop",
            "cohort_id",
            "cohort_label",
            "statistic",
            "delta_i",
        ]
    ]

    # Add focal_region column.
    df_signals = df_signals.assign(
        focal_region=df_signals["contig"]
        + " ( "
        + df_signals["focus_pstart"].apply(
            lambda x: "{:,}".format(
                x,
            )
        )
        + " - "
        + df_signals["focus_pstop"].apply(
            lambda x: "{:,}".format(
                x,
            )
        )
        + " )"
    )

    # Add url column.
    df_signals["url"] = "../cohort/" + df_signals["cohort_id"] + ".html"

    def make_clickable(x):
        url = x["url"]
        name = x["cohort_label"]
        return '<a href="{}" rel="noopener noreferrer" target="_blank">{}</a>'.format(
            url, name
        )

    # Add hyperlinked cohort_label column
    df_signals["cohort_label"] = df_signals.apply(make_clickable, axis=1)

    # Style the table.
    styled = (
        df_signals[["focal_region", "cohort_label", "statistic", "delta_i"]]
        .rename(
            columns={
                "focal_region": "Focal Region",
                "cohort_label": "Cohort Label",
                "statistic": "Statistic",
                "delta_i": "Delta i",
            }
        )
        .style.hide(axis="index")
    )

    return styled


style_signals_table(df_signals)